# NPRG030 Programování I – cvičení 27.11.2024

- motivace: 

## Stromy
### Problémy
   * metoda, ktera vrati maximalni prvek (v obecnem strome tak v BVS)
   * metoda, ktera zreflektuje strom podle svisle osy
   * metoda, ktera vrati, kolik je ve stromu cisel v rozmezi (a,b), strom je BVS
   * metoda evaluace stromu

In [2]:
class Node:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right


class TreeSet:
    def __init__(self, root=None):
        self.root = root
    
    def max(self) -> int | None:
        """Returns biggest value in the tree-set, None if empty."""
        if self.root is None:
            return None
        return self._max(self.root, 0)

    def _max(self, n: Node, m: int) -> int | None:
        """Returns the biggest value node in subtree `n`."""
        if n is None:
            return m
        m = max(
            m,
            n.val,
            self._max(n.left, m),
            self._max(n.right, m)
        )
        return m

    def maxBST(self) -> int | None:
        """Returns biggest value in the tree-set, None if empty."""
        if self.root is None:
            return None
        return self._maxBST(self.root).val

    def _maxBST(self, n: Node) -> Node:
        """Returns the biggest value node in subtree `n`."""
        if n.right is None:
            return n
        return self._maxBST(n.right)
    
    def countBST(self, lo: int, hi: int) -> int:
        """Returns number of values x in the tree-set such that lo <= x <= hi."""

        def rec_count(n) -> int:
            if n is None:
                return 0
            if n.val < lo:
                return rec_count(n.right)
            if n.val > hi:
                return rec_count(n.left)
            # else this must be the case: lo <= n.val <= hi
            return 1 + rec_count(n.left) + rec_count(n.right)

        return rec_count(self.root)

        
def tree(a: list, x: int, y: int):
    """builds a tree :)"""
    if x > y:
        return None
    p = Node(a[(x+y)//2])
    p.left = tree(a, x, (x + y) // 2 - 1)
    p.right = tree(a, (x + y) // 2 + 1, y)
    return p

#####################################

l1 = [1,2,4,8,9,11,12,13,20,25,50,100]
l1.sort()
t1 = TreeSet(tree(l1, 0, len(l1) - 1))

#------------------------------------
print("=" * (len(l1) * 4))
print(f"{l1=}")
print("=" * (len(l1) * 4))

print(f"{t1.max() = }\n\tshould be ={max(l1)}")
print(f"{t1.countBST(21, 24) = }")



l1=[1, 2, 4, 8, 9, 11, 12, 13, 20, 25, 50, 100]
t1.max() = 100
	should be =100
t1.countBST(21, 24) = 0


### Binarni strom
 - hodnoty jsou binarni operatory
 - listy jsou cisla

 - implementuj metodu eval(), ktera vyhodnoti strom

In [3]:
expr = TreeSet(Node("+", left=Node("*", Node(2), Node(6)), right=Node("-", Node("/", Node(8), Node(4)), Node(6))))
"""
                    +
        *                       -
    2       6               /       6
                        8       4
((2*6) + ((8/4) - 6)) = 8
"""
def eval(t: TreeSet) -> int:
    def _eval(n):
        if n.right is None: # n is a leaf:
            return n.val
        
        if n.val == "+":
            return _eval(n.left) + _eval(n.right)
        elif n.val == "-":
            return _eval(n.left) - _eval(n.right)
        elif n.val == "*":
            return _eval(n.left) * _eval(n.right)
        elif n.val == "/":
            return _eval(n.left) / _eval(n.right)
        else:
            assert False, "INVALID OPERATOR, use +,-,*,/"
    
    return _eval(t.root)

print(eval(expr))
        

8.0


- python neco podobneho... module AST - Abstract Syntax Trees

# Prohledávání stavového prostoru

 - jednoduše převedeme na prohledávání grafu
    - vím, jak generovat sousedy aktuální pozice
 
 - bludiste
    - pozice `(x, y)` ==sousedi==> `(x+1, y)`, `(x-1, y)`, `(x, y+1)`, `(x, y-1)`

### Problem: bludiste
```
  ######
# #    #
# ##   #
#  # # #
#    # #
######  
```
- v bludisti najit nejkratsi cestu:
```
**######
#*#    #
#*##***#
#**#*#*#
# ***#*#
######* 
```
 - ... tedy prohledavani do sirky = fronta

In [74]:
from collections import deque


def read_maze():
    print("enter the maze:")
    maze = []
    while True:
        line = input()
        if not line:
            break
        maze.append(list(line))
    return maze

def shortest_path(maze, start, end):
    rows, cols = len(maze), len(maze[0])
    queue = deque([(start, [])])
    visited = set()
    
    while queue:
        (x, y), path = queue.popleft()
        if (x, y) == end:
            return path + [(x, y)]
        
        if (x, y) in visited:
            continue
        visited.add((x, y))
        
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < rows and 0 <= ny < cols and maze[nx][ny] == ' ' and (nx, ny) not in visited:
                print(nx, ny)
                queue.append(((nx, ny), path + [(x, y)]))
    return None

# maze = read_maze()
string = """  ######
# #    #
# ##   #
#  #   #
#      #
######  """
maze = [[char for char in row] for row in string.split('\n')]
start, end = (0, 0), (len(maze)-1, len(maze[0])-1)
print(*maze, sep="\n")
print(start, end)
print(shortest_path(maze, (0, 0), (len(maze), len(maze[0]))))



[' ', ' ', '#', '#', '#', '#', '#', '#']
['#', ' ', '#', ' ', ' ', ' ', ' ', '#']
['#', ' ', '#', '#', ' ', ' ', ' ', '#']
['#', ' ', ' ', '#', ' ', ' ', ' ', '#']
['#', ' ', ' ', ' ', ' ', ' ', ' ', '#']
['#', '#', '#', '#', '#', '#', ' ', ' ']
(0, 0) (5, 7)
0 0
0 1
0 1
1 1
1 1
2 1
2 1
3 1
3 1
4 1
3 2
4 1
4 2
3 2
4 2
4 2
4 3
4 2
4 3
4 4
4 4
3 4
4 5
3 4
2 4
3 5
4 5
3 5
4 6
2 4
1 4
2 5
3 5
2 5
3 6
3 5
4 6
3 6
5 6
1 4
1 3
1 5
2 5
1 5
2 6
2 5
3 6
2 6
3 6
5 6
5 7
1 3
1 5
1 6
1 5
2 6
1 6
2 6
5 7
1 6
1 6
None


### DEKOMPOZICE:
 - nacteni vstupu
 - hledani cesty
 - fronta 
